In [1]:
df = spark.read.csv('/FileStore/tables/heart.csv',sep=',',inferSchema='true',header='true')

In [2]:
df.show(1)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
 63| 1| 3| 145| 233| 1| 0| 150| 0| 2.3| 0| 0| 1| 1|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
only showing top 1 row

In [3]:
from pyspark.ml.feature import VectorAssembler

In [4]:
feat_cols = ['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

In [5]:
vec_assember = VectorAssembler(inputCols = feat_cols, outputCol='features' )

In [6]:
final_data = vec_assember.transform(df)

In [7]:
final_data.take(2)

Out[11]: [Row(age=63, sex=1, cp=3, trestbps=145, chol=233, fbs=1, restecg=0, thalach=150, exang=0, oldpeak=2.3, slope=0, ca=0, thal=1, target=1, features=DenseVector([63.0, 1.0, 3.0, 145.0, 233.0, 1.0, 0.0, 150.0, 0.0, 2.3, 0.0, 0.0, 1.0])),
 Row(age=37, sex=1, cp=2, trestbps=130, chol=250, fbs=0, restecg=1, thalach=187, exang=0, oldpeak=3.5, slope=0, ca=0, thal=2, target=1, features=DenseVector([37.0, 1.0, 2.0, 130.0, 250.0, 0.0, 1.0, 187.0, 0.0, 3.5, 0.0, 0.0, 2.0]))]

In [8]:
training,testing = final_data.randomSplit([0.7,0.3],seed=42)

In [9]:
from pyspark.ml.classification import LogisticRegression

In [10]:
lr = LogisticRegression(labelCol='target',featuresCol='features')

In [11]:
lrModel = lr.fit(training)

In [12]:
predictions = lrModel.transform(testing)

In [13]:
predictions.select('target','prediction','probability','age','sex').show()

+------+----------+--------------------+---+---+
target|prediction| probability|age|sex|
+------+----------+--------------------+---+---+
 1| 1.0|[0.02332773515992...| 34| 1|
 1| 1.0|[0.08090348837775...| 35| 1|
 1| 1.0|[0.36018516108440...| 37| 1|
 1| 0.0|[0.52121152577901...| 38| 1|
 1| 1.0|[0.03356703008638...| 39| 0|
 0| 0.0|[0.75379138306753...| 39| 1|
 1| 1.0|[0.09877451547508...| 39| 1|
 1| 1.0|[0.01989569567949...| 41| 0|
 1| 1.0|[0.03963092985326...| 41| 0|
 0| 1.0|[0.37342758878227...| 41| 1|
 1| 1.0|[0.10774081636666...| 41| 1|
 1| 1.0|[0.24595120154369...| 41| 1|
 1| 1.0|[0.18754278378722...| 42| 0|
 1| 1.0|[0.01137511701886...| 42| 0|
 1| 1.0|[0.21846498051137...| 42| 1|
 1| 1.0|[0.19741161160178...| 42| 1|
 1| 1.0|[0.08272586814728...| 42| 1|
 1| 1.0|[0.31878224996606...| 43| 1|
 0| 0.0|[0.99396095207488...| 43| 1|
 1| 1.0|[0.45994059950948...| 43| 1|
+------+----------+--------------------+---+---+
only showing top 20 rows

In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [15]:
evaluator = BinaryClassificationEvaluator(labelCol='target')

In [16]:
evaluator.evaluate(predictions)

Out[20]: 0.9244897959183674